<h1>410821322 Steven Jonathan Machine Learning Homework 4</h1>

<h2> First setup </h2>

<h3>Removing excessive lines at the end of horse-colic.data</h3>

In [1]:
line = open('horse-colic.data', 'r', encoding='utf-8').readlines() #Remove the endspace so that fitting won't be a problem
temp = []
for i in line:
    i = i.strip()
    temp.append(i+'\n')
line = open('horse-colic.data', 'w', encoding='utf-8')
line.writelines(temp)
line.close()

<h3>Removing excessive lines at the end of horse-colic.test</h3>

In [2]:
line = open('horse-colic.test', 'r', encoding='utf-8').readlines() #Remove the endspace so that fitting won't be a problem
temp = []
for i in line:
    i = i.strip()
    temp.append(i+'\n')
line = open('horse-colic.test', 'w', encoding='utf-8')
line.writelines(temp)
line.close()

<h2> Setting up data for KNNImputer </h2>

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

#Setting up .data
headernames = ["Surgery?","Age","HospitalNo","RectalTemp","Pulse","RespiratoryRate","ExtremitiesTemp","PeripheralPulse","MucousMembranes","CapillaryRefillTime","PainLevel","Peristalsis","AbdominalDistension","NasogastricTube","NasogastricReflux","NasogastricRefluxPH","Feces","Abdomen","PackedCellVolume","TotalProtein","AbdominocentesisAppearance","AbdominocentesisTotalProtein","Outcome","SurgicalLesion","LesionTypeOne","LesionTypeTwo","LesionTypeThree","CPData"]
horsedataoriginal = pd.read_csv("horse-colic.data",sep=" ",index_col=False,names=headernames).replace("?", np.NaN) #change the data file path, also replace ? with NaN for easier process
horsedataoriginal = horsedataoriginal.iloc[:,:23] #We don't use 23 further, remove the factors

horsedataoriginal = horsedataoriginal.drop(columns=["HospitalNo"]) #Remove unused factors
horsedataoriginal["Outcome"] = horsedataoriginal["Outcome"].replace("3","2") #Replace Euthanized with dead since it'll be dead anyway

horsedatatest = pd.read_csv("horse-colic.test",sep=" ",index_col=False,names=headernames).replace('?', np.NaN)#change the data file path, also replace ? with NaN for easier process
horsedatatest = horsedatatest.iloc[:,:23]
horsedatatest = horsedatatest.drop(columns=["HospitalNo"]) #Remove unused factors
horsedatatest["Outcome"] = horsedatatest["Outcome"].replace("3","2")

In [4]:
display(horsedataoriginal)

,Surgery?,Age,RectalTemp,Pulse,RespiratoryRate,ExtremitiesTemp,PeripheralPulse,MucousMembranes,CapillaryRefillTime,PainLevel,...,NasogastricTube,NasogastricReflux,NasogastricRefluxPH,Feces,Abdomen,PackedCellVolume,TotalProtein,AbdominocentesisAppearance,AbdominocentesisTotalProtein,Outcome
0,2,1,38.50,66,28,3,3,NaN,2,5,...,NaN,NaN,NaN,3,5,45.00,8.40,NaN,NaN,2
1,1,1,39.2,88,20,NaN,NaN,4,1,3,...,NaN,NaN,NaN,4,2,50,85,2,2,2
2,2,1,38.30,40,24,1,1,3,1,3,...,NaN,NaN,NaN,1,1,33.00,6.70,NaN,NaN,1
3,1,9,39.10,164,84,4,1,6,2,2,...,1,2,5.00,3,NaN,48.00,7.20,3,5.30,2
4,2,1,37.30,104,35,NaN,NaN,6,2,NaN,...,NaN,NaN,NaN,NaN,NaN,74.00,7.40,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,1,NaN,120,70,4,NaN,4,2,2,...,NaN,NaN,NaN,NaN,5,55,65,NaN,NaN,2
296,2,1,37.20,72,24,3,2,4,2,4,...,3,1,NaN,4,4,44.00,NaN,3,3.30,2
297,1,1,37.50,72,30,4,3,4,1,4,...,2,1,NaN,3,5,60.00,6.80,NaN,NaN,2
298,1,1,36.50,100,24,3,3,3,1,3,...,3,1,NaN,4,4,50.00,6.00,3,3.40,1


In [5]:
display(horsedatatest)

,Surgery?,Age,RectalTemp,Pulse,RespiratoryRate,ExtremitiesTemp,PeripheralPulse,MucousMembranes,CapillaryRefillTime,PainLevel,...,NasogastricTube,NasogastricReflux,NasogastricRefluxPH,Feces,Abdomen,PackedCellVolume,TotalProtein,AbdominocentesisAppearance,AbdominocentesisTotalProtein,Outcome
0,2,1,38.50,54,20,NaN,1,2,2,3,...,2,2,5.90,NaN,2,42.00,6.30,NaN,NaN,1
1,2,1,37.60,48,36,NaN,NaN,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,44.00,6.30,1,5.00,1
2,1,1,37.7,44,28,NaN,4,3,2,5,...,1,1,NaN,3,5,45,70,3,2,1
3,1,1,37,56,24,3,1,4,2,4,...,1,1,NaN,NaN,NaN,35,61,3,2,2
4,2,1,38.00,42,12,3,NaN,3,1,1,...,NaN,NaN,NaN,NaN,2,37.00,5.80,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2,1,38.60,60,30,1,1,3,1,4,...,1,1,NaN,NaN,NaN,40.00,6.00,1,NaN,1
64,2,1,37.80,42,40,1,1,1,1,1,...,NaN,NaN,NaN,3,3,36.00,6.20,NaN,NaN,1
65,1,1,38,60,12,1,1,2,1,2,...,1,1,NaN,1,4,44,65,3,2,2
66,2,1,38.00,42,12,3,NaN,3,1,1,...,NaN,NaN,NaN,NaN,1,37.00,5.80,NaN,NaN,1


In [6]:
countnadata = (horsedataoriginal.isna()).sum().sum()
print(f"Count for missing data in the data now: {countnadata}")
countnatest = (horsedatatest.isna()).sum().sum()
print(f"Count for missing data in the testnow: {countnatest}")

Count for missing data in the data now: 1605
Count for missing data in the testnow: 322


In [7]:
horsedata = horsedataoriginal
horsetest = horsedatatest

<h2>Creating the KNNImputer</h2>


In [8]:
Kimputer = KNNImputer(n_neighbors=20)
imputedhorsedata = pd.DataFrame(Kimputer.fit_transform(horsedata))

imputedhorsetest = pd.DataFrame(Kimputer.fit_transform(horsetest))

In [9]:
display(imputedhorsedata[:22])

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,2.0,1.0,38.500,66.0,28.00,3.0,3.00,2.65,2.0,5.00,...,1.95,1.40,5.645,3.00,5.0,45.000,8.400,1.85,3.875,2.0
1,1.0,1.0,39.200,88.0,20.00,2.6,2.50,4.00,1.0,3.00,...,1.95,1.50,3.490,4.00,2.0,50.000,85.000,2.00,2.000,2.0
2,2.0,1.0,38.300,40.0,24.00,1.0,1.00,3.00,1.0,3.00,...,1.75,1.30,5.470,1.00,1.0,33.000,6.700,1.65,3.860,1.0
3,1.0,9.0,39.100,164.0,84.00,4.0,1.00,6.00,2.0,2.00,...,1.00,2.00,5.000,3.00,4.3,48.000,7.200,3.00,5.300,2.0
4,2.0,1.0,37.300,104.0,35.00,3.0,3.00,6.00,2.0,3.90,...,1.55,2.05,5.345,3.35,4.5,74.000,7.400,2.60,3.730,2.0
5,2.0,1.0,38.260,56.2,24.60,2.0,1.00,3.00,1.0,2.00,...,2.00,1.00,4.935,3.00,3.0,39.275,24.145,1.60,3.030,1.0
6,1.0,1.0,37.900,48.0,16.00,1.0,1.00,1.00,1.0,3.00,...,1.00,1.00,5.435,3.00,5.0,37.000,7.000,1.60,3.695,1.0
7,1.0,1.0,38.220,60.0,34.15,3.0,2.00,2.90,1.0,3.05,...,2.00,1.00,5.495,3.00,4.0,44.000,8.300,1.60,3.975,2.0
8,2.0,1.0,38.140,80.0,36.00,3.0,4.00,3.00,1.0,4.00,...,2.00,1.00,5.390,3.00,5.0,38.000,6.200,2.05,3.705,2.0
9,2.0,9.0,38.300,90.0,35.40,1.0,1.85,1.00,1.0,5.00,...,2.00,1.00,5.575,3.00,4.1,40.000,6.200,1.00,2.200,1.0


In [10]:
display(imputedhorsetest)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,2.0,1.0,38.5,54.0,20.0,2.25,1.0,2.0,2.0,3.0,...,2.00,2.00,5.90000,3.05,2.0,42.0,6.3,1.85,2.805,1.0
1,2.0,1.0,37.6,48.0,36.0,2.05,1.3,1.0,1.0,2.3,...,1.75,1.65,5.80625,3.10,3.3,44.0,6.3,1.00,5.000,1.0
2,1.0,1.0,37.7,44.0,28.0,2.20,4.0,3.0,2.0,5.0,...,1.00,1.00,5.80625,3.00,5.0,45.0,70.0,3.00,2.000,1.0
3,1.0,1.0,37.0,56.0,24.0,3.00,1.0,4.0,2.0,4.0,...,1.00,1.00,5.80625,2.95,4.3,35.0,61.0,3.00,2.000,2.0
4,2.0,1.0,38.0,42.0,12.0,3.00,1.2,3.0,1.0,1.0,...,1.75,1.65,5.80625,3.15,2.0,37.0,5.8,1.85,2.675,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2.0,1.0,38.6,60.0,30.0,1.00,1.0,3.0,1.0,4.0,...,1.00,1.00,5.80625,3.15,3.7,40.0,6.0,1.00,2.910,1.0
64,2.0,1.0,37.8,42.0,40.0,1.00,1.0,1.0,1.0,1.0,...,1.75,1.65,5.80625,3.00,3.0,36.0,6.2,1.95,2.785,1.0
65,1.0,1.0,38.0,60.0,12.0,1.00,1.0,2.0,1.0,2.0,...,1.00,1.00,5.80625,1.00,4.0,44.0,65.0,3.00,2.000,2.0
66,2.0,1.0,38.0,42.0,12.0,3.00,1.2,3.0,1.0,1.0,...,1.75,1.65,5.80625,3.15,1.0,37.0,5.8,1.85,2.675,1.0


In [11]:
countnadata = (imputedhorsedata.isna()).sum().sum()
print(f"Count for missing data in the data now: {countnadata}")
countnatest = (imputedhorsetest.isna()).sum().sum()
print(f"Count for missing data in the test now: {countnatest}")

Count for missing data in the data now: 0
Count for missing data in the testnow: 0


In [12]:
x_train = imputedhorsedata.loc[:,:20].astype(float)
y_train = round(imputedhorsedata[21],0) #If no round then error - I dunno why
x_test = imputedhorsetest.loc[:,:20].astype(float)
y_test = round(imputedhorsetest[21],0) #These too

<h2>Creating the KNeighborsClassifier</h2>

First, we try to see which K nearest neighbor is the best for this prediction 

In [13]:
highest = 0
highestindex = 1
for i in range(1,150):
    Kclassifier = KNeighborsClassifier(n_neighbors=i)
    Kclassifier.fit(x_train,y_train)
    y_pred = Kclassifier.predict(x_test)
    if(round(accuracy_score(y_test, y_pred)*100,2)>highest):
        highest = round(accuracy_score(y_test, y_pred)*100,2)
        highestindex=i
        print("Accuracy: ",round(accuracy_score(y_test, y_pred)*100,2),"%","in i = ",i)
    
print(f"Highest Accuracy at: {highest}% least neighbors at : {highestindex}")

Accuracy:  67.65 % in i =  1
Accuracy:  75.0 % in i =  2
Accuracy:  76.47 % in i =  9
Accuracy:  77.94 % in i =  16
Accuracy:  79.41 % in i =  128
Accuracy:  80.88 % in i =  140
Accuracy:  82.35 % in i =  142
Highest Accuracy at: 82.35% least neighbors at : 142


Here we can see that as the neighbor goes bigger, the accuracy will be less, therefore we can take the highest accuracy in which K-Neighbors, which is 82.35% when we set the n_neighbors = 142

In [14]:
Kclassifier = KNeighborsClassifier(n_neighbors=142,metric='euclidean')
Kclassifier.fit(x_train,y_train)
y_pred = Kclassifier.predict(x_test)
print("Accuracy: ",round(accuracy_score(y_test, y_pred)*100,2),"%")

Accuracy:  82.35 %
